#### importing pandas library

In [1]:
import pandas as pd

In [2]:
import boto3
import s3fs

In [3]:
import os

#### Assigning taxi dataset 

In [4]:
client = boto3.client('s3')

In [5]:
path = 's3://nyc-taxi-system/Sample_dataset.csv'

In [6]:
df1 = pd.read_csv(path)
df1.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,0,1.0,2020-01-07 21:38:40,2020-01-07 21:45:13,2.0,1.00,1.0,N,249,113,1.0,6.5,3.0,0.5,2.55,0.0,0.3,12.85,2.5
1,1,2.0,2020-01-24 20:09:35,2020-01-24 20:52:36,1.0,3.99,1.0,N,144,142,1.0,26.0,0.5,0.5,5.96,0.0,0.3,35.76,2.5
2,2,2.0,2020-01-15 13:37:54,2020-01-15 13:40:47,1.0,0.43,1.0,N,238,238,1.0,4.0,0.0,0.5,2.19,0.0,0.3,9.49,2.5
3,3,2.0,2020-01-16 16:22:45,2020-01-16 16:28:19,1.0,0.60,1.0,N,164,137,1.0,5.5,1.0,0.5,1.00,0.0,0.3,10.80,2.5
4,4,1.0,2020-01-01 14:11:40,2020-01-01 14:12:28,4.0,0.00,1.0,N,132,132,4.0,2.5,0.0,0.5,0.00,0.0,0.3,3.30,0.0


#### Assigning taxi zones dataset

In [7]:
path = 's3://nyc-taxi-system/Taxi_Zone_Lookup_Fixed.csv'

In [8]:
df2 = pd.read_csv('./Taxi_Zone_Lookup_Fixed.csv')
df2.head()

,Unnamed: 0,X,Y,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,latitude,longitude
0,0,9.352230e+05,190535.0526,1,0.116357,0.000782,Newark Airport,1,EWR,40.689064,-74.177255
1,1,1.032516e+06,167292.4932,2,0.433470,0.004866,Jamaica Bay,2,Queens,40.603994,-73.835412
2,2,1.025883e+06,254779.6006,3,0.084341,0.000314,Allerton,3,Bronx,40.863141,-73.861346
3,3,9.906188e+05,203105.5323,4,0.043567,0.000112,Alphabet City,4,Manhattan,40.725102,-73.979583
4,4,9.314680e+05,139837.4784,5,0.092146,0.000498,Arden Heights,5,Staten Island,40.556496,-74.173875


In [9]:
df2 = df2.drop(['Unnamed: 0'], axis=1)

In [10]:
df2.columns

Index(['X', 'Y', 'OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'LocationID',
       'borough', 'latitude', 'longitude'],
      dtype='object')

#### Dataset for pick up 

In [11]:
df3 = df2.drop(['X', 'Y', 'OBJECTID', 'Shape_Leng', 'Shape_Area'], axis=1)
df3 = df3.rename(columns={"zone":"PU_zone", "LocationID":"PULocationID", "borough":"PU_borough", "latitude":"PU_latitude", "longitude":"PU_longitude"})
df3.head()

,PU_zone,PULocationID,PU_borough,PU_latitude,PU_longitude
0,Newark Airport,1,EWR,40.689064,-74.177255
1,Jamaica Bay,2,Queens,40.603994,-73.835412
2,Allerton,3,Bronx,40.863141,-73.861346
3,Alphabet City,4,Manhattan,40.725102,-73.979583
4,Arden Heights,5,Staten Island,40.556496,-74.173875


#### Dataset for drop off

In [12]:
df4 = df2.drop(['X', 'Y', 'OBJECTID', 'Shape_Leng', 'Shape_Area'], axis=1)
df4 = df4.rename(columns={"zone":"DO_zone", "LocationID":"DOLocationID", "borough":"DO_borough", "latitude":"DO_latitude", "longitude":"DO_longitude"})
df4.head()

,DO_zone,DOLocationID,DO_borough,DO_latitude,DO_longitude
0,Newark Airport,1,EWR,40.689064,-74.177255
1,Jamaica Bay,2,Queens,40.603994,-73.835412
2,Allerton,3,Bronx,40.863141,-73.861346
3,Alphabet City,4,Manhattan,40.725102,-73.979583
4,Arden Heights,5,Staten Island,40.556496,-74.173875


#### Using left join for pick-up location

In [13]:
df5 = pd.merge(df1, df3, how="left", on=["PULocationID", "PULocationID"])
df5.head(20)

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,PU_zone,PU_borough,PU_latitude,PU_longitude
0,0,1.0,2020-01-07 21:38:40,2020-01-07 21:45:13,2.0,1.00,1.0,N,249,113,...,0.5,2.55,0.00,0.3,12.85,2.5,West Village,Manhattan,40.734186,-74.005580
1,1,2.0,2020-01-24 20:09:35,2020-01-24 20:52:36,1.0,3.99,1.0,N,144,142,...,0.5,5.96,0.00,0.3,35.76,2.5,Little Italy/NoLiTa,Manhattan,40.721006,-73.994702
2,2,2.0,2020-01-15 13:37:54,2020-01-15 13:40:47,1.0,0.43,1.0,N,238,238,...,0.5,2.19,0.00,0.3,9.49,2.5,Upper West Side North,Manhattan,40.781960,-73.981243
3,3,2.0,2020-01-16 16:22:45,2020-01-16 16:28:19,1.0,0.60,1.0,N,164,137,...,0.5,1.00,0.00,0.3,10.80,2.5,Midtown South,Manhattan,40.749842,-73.984251
4,4,1.0,2020-01-01 14:11:40,2020-01-01 14:12:28,4.0,0.00,1.0,N,132,132,...,0.5,0.00,0.00,0.3,3.30,0.0,JFK Airport,Queens,40.642948,-73.779373
5,5,2.0,2020-01-22 23:33:47,2020-01-22 23:39:45,2.0,1.45,1.0,N,142,237,...,0.5,1.62,0.00,0.3,12.42,2.5,Lincoln Square East,Manhattan,40.775707,-73.991420
6,6,1.0,2020-01-04 16:12:29,2020-01-04 16:31:23,2.0,3.90,1.0,N,90,239,...,0.5,3.50,0.00,0.3,23.30,2.5,Flatiron,Manhattan,40.741059,-73.989642
7,7,1.0,2020-01-31 18:49:37,2020-01-31 19:03:00,1.0,2.90,1.0,N,230,236,...,0.5,0.00,0.00,0.3,15.80,2.5,Times Sq/Theatre District,Manhattan,40.758168,-73.985525
8,8,2.0,2020-01-12 07:09:42,2020-01-12 07:14:05,2.0,1.19,1.0,N,263,140,...,0.5,1.86,0.00,0.3,11.16,2.5,Yorkville West,Manhattan,43.670470,-79.387252
9,9,2.0,2020-01-01 10:49:09,2020-01-01 10:52:21,1.0,0.77,1.0,N,42,42,...,0.5,0.00,0.00,0.3,5.30,0.0,Central Harlem North,Manhattan,40.799218,-73.954900


#### Using left join for drop off location

In [14]:
df6 = pd.merge(df5, df4, how="left", on=["DOLocationID", "DOLocationID"])
df6 = df6.drop(['Unnamed: 0'], axis=1)
df6.head(20)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,PU_zone,PU_borough,PU_latitude,PU_longitude,DO_zone,DO_borough,DO_latitude,DO_longitude
0,1.0,2020-01-07 21:38:40,2020-01-07 21:45:13,2.0,1.00,1.0,N,249,113,1.0,...,12.85,2.5,West Village,Manhattan,40.734186,-74.005580,Greenwich Village North,Manhattan,40.734299,-74.000573
1,2.0,2020-01-24 20:09:35,2020-01-24 20:52:36,1.0,3.99,1.0,N,144,142,1.0,...,35.76,2.5,Little Italy/NoLiTa,Manhattan,40.721006,-73.994702,Lincoln Square East,Manhattan,40.775707,-73.991420
2,2.0,2020-01-15 13:37:54,2020-01-15 13:40:47,1.0,0.43,1.0,N,238,238,1.0,...,9.49,2.5,Upper West Side North,Manhattan,40.781960,-73.981243,Upper West Side North,Manhattan,40.781960,-73.981243
3,2.0,2020-01-16 16:22:45,2020-01-16 16:28:19,1.0,0.60,1.0,N,164,137,1.0,...,10.80,2.5,Midtown South,Manhattan,40.749842,-73.984251,Kips Bay,Manhattan,40.739546,-73.977083
4,1.0,2020-01-01 14:11:40,2020-01-01 14:12:28,4.0,0.00,1.0,N,132,132,4.0,...,3.30,0.0,JFK Airport,Queens,40.642948,-73.779373,JFK Airport,Queens,40.642948,-73.779373
5,2.0,2020-01-22 23:33:47,2020-01-22 23:39:45,2.0,1.45,1.0,N,142,237,1.0,...,12.42,2.5,Lincoln Square East,Manhattan,40.775707,-73.991420,Upper East Side South,Manhattan,40.779468,-73.958265
6,1.0,2020-01-04 16:12:29,2020-01-04 16:31:23,2.0,3.90,1.0,N,90,239,1.0,...,23.30,2.5,Flatiron,Manhattan,40.741059,-73.989642,Upper West Side South,Manhattan,40.769244,-73.990767
7,1.0,2020-01-31 18:49:37,2020-01-31 19:03:00,1.0,2.90,1.0,N,230,236,2.0,...,15.80,2.5,Times Sq/Theatre District,Manhattan,40.758168,-73.985525,Upper East Side North,Manhattan,40.764873,-73.961511
8,2.0,2020-01-12 07:09:42,2020-01-12 07:14:05,2.0,1.19,1.0,N,263,140,1.0,...,11.16,2.5,Yorkville West,Manhattan,43.670470,-79.387252,Lenox Hill East,Manhattan,40.765177,-73.961595
9,2.0,2020-01-01 10:49:09,2020-01-01 10:52:21,1.0,0.77,1.0,N,42,42,2.0,...,5.30,0.0,Central Harlem North,Manhattan,40.799218,-73.954900,Central Harlem North,Manhattan,40.799218,-73.954900


#### finalizing dataset

In [15]:
from secrets import access_key, secret_access_key

In [16]:
client = boto3.client('s3',
                        aws_access_key_id = access_key,
                        aws_secret_access_key = secret_access_key)

In [17]:
df6.to_csv('D:/Project/Main/Final_dataset.csv')

In [18]:
df6.shape

(1200911, 26)

In [19]:
client.upload_file('D:/Project/Main/Final_dataset.csv', 'nyc-taxi-system', 'Final_dataset.csv')
print("Uploaded on AWS")
os.remove('D:/Project/Main/Final_dataset.csv')
print("File removed")

Uploaded on AWS
File removed
